In [1]:
!pip install duckdb -q

In [2]:
import pandas as pd
import duckdb

https://techtfq.com/blog/learn-how-to-write-sql-queries-practice-complex-sql-queries


### 1 - Write a SQL query to fetch all the duplicate records from a table.

In [3]:
duckdb.query("""
            create table users
            (
            user_id int primary key,
            user_name varchar(30) not null,
            email varchar(50));
""")

In [4]:
duckdb.query("""
            insert into users values
            (1, 'Sumit', 'sumit@gmail.com'),
            (2, 'Reshma', 'reshma@gmail.com'),
            (3, 'Farhana', 'farhana@gmail.com'),
            (4, 'Robin', 'robin@gmail.com'),
            (5, 'Robin', 'robin@gmail.com');
""")

In [5]:
duckdb.query("""
            SELECT * FROM USERS
""").df()

,user_id,user_name,email
0,1,Sumit,sumit@gmail.com
1,2,Reshma,reshma@gmail.com
2,3,Farhana,farhana@gmail.com
3,4,Robin,robin@gmail.com
4,5,Robin,robin@gmail.com


In [6]:
duckdb.query("""
            SELECT * FROM (SELECT *,
                           ROW_NUMBER () OVER (PARTITION BY USER_NAME ORDER BY USER_ID) AS RN
                           FROM USERS) AS t1
            WHERE RN > 1                 
""").df()

,user_id,user_name,email,RN
0,5,Robin,robin@gmail.com,2


### 2 - Write a SQL query to fetch the second last record from a employee table.

In [7]:
duckdb.query("""
            create table employee
            ( emp_ID int primary key
            ,emp_NAME varchar(50) not null
            ,DEPT_NAME varchar(50)
            ,SALARY int);                 
""")

In [8]:
duckdb.query("""
            insert into employee values(101, 'Mohan', 'Admin', 4000);
            insert into employee values(102, 'Rajkumar', 'HR', 3000);
            insert into employee values(103, 'Akbar', 'IT', 4000);
            insert into employee values(104, 'Dorvin', 'Finance', 6500);
            insert into employee values(105, 'Rohit', 'HR', 3000);
            insert into employee values(106, 'Rajesh',  'Finance', 5000);
            insert into employee values(107, 'Preet', 'HR', 7000);
            insert into employee values(108, 'Maryam', 'Admin', 4000);
            insert into employee values(109, 'Sanjay', 'IT', 6500);
            insert into employee values(110, 'Vasudha', 'IT', 7000);
            insert into employee values(111, 'Melinda', 'IT', 8000);
            insert into employee values(112, 'Komal', 'IT', 10000);
            insert into employee values(113, 'Gautham', 'Admin', 2000);
            insert into employee values(114, 'Manisha', 'HR', 3000);
            insert into employee values(115, 'Chandni', 'IT', 4500);
            insert into employee values(116, 'Satya', 'Finance', 6500);
            insert into employee values(117, 'Adarsh', 'HR', 3500);
            insert into employee values(118, 'Tejaswi', 'Finance', 5500);
            insert into employee values(119, 'Cory', 'HR', 8000);
            insert into employee values(120, 'Monica', 'Admin', 5000);
            insert into employee values(121, 'Rosalin', 'IT', 6000);
            insert into employee values(122, 'Ibrahim', 'IT', 8000);
            insert into employee values(123, 'Vikram', 'IT', 8000);
            insert into employee values(124, 'Dheeraj', 'IT', 11000);
""")

In [9]:
duckdb.query("""
            SELECT * FROM (SELECT *,
                           ROW_NUMBER () OVER (ORDER BY EMP_ID DESC) AS RN
                           FROM EMPLOYEE) AS t1
            WHERE RN = 2
""").df()

,emp_ID,emp_NAME,DEPT_NAME,SALARY,RN
0,123,Vikram,IT,8000,2


### 3 - Write a SQL query to display only the details of employees who either earn the highest salary or the lowest salary in each department from the employee table.

In [10]:
duckdb.query("""
            WITH t1 AS (SELECT *,
                        RANK () OVER (PARTITION BY DEPT_NAME ORDER BY SALARY DESC) AS RNK_MAX
                        FROM EMPLOYEE),
                 t2 AS (SELECT *,
                        RANK () OVER (PARTITION BY DEPT_NAME ORDER BY SALARY ASC) AS RNK_MIN
                        FROM EMPLOYEE)
                        
            SELECT t1.EMP_ID, t1.EMP_NAME, t1.DEPt_NAME, t1.SALARY
            FROM t1
            JOIN t2
            ON t1.EMP_ID = t2.EMP_ID
            WHERE RNK_MAX = 1 OR RNK_MIN = 1
            ORDER BY t1.DEPT_NAME
""").df()

,emp_ID,emp_NAME,DEPT_NAME,SALARY
0,120,Monica,Admin,5000
1,113,Gautham,Admin,2000
2,104,Dorvin,Finance,6500
3,116,Satya,Finance,6500
4,106,Rajesh,Finance,5000
5,119,Cory,HR,8000
6,102,Rajkumar,HR,3000
7,105,Rohit,HR,3000
8,114,Manisha,HR,3000
9,124,Dheeraj,IT,11000


### 4 - From the doctors table, fetch the details of doctors who work in the same hospital but in different specialty.

In [11]:
duckdb.query("""
            create table doctors
            (
            id int primary key,
            name varchar(50) not null,
            speciality varchar(100),
            hospital varchar(50),
            city varchar(50),
            consultation_fee int
            );
""")

In [12]:
duckdb.query("""
            insert into doctors values
            (1, 'Dr. Shashank', 'Ayurveda', 'Apollo Hospital', 'Bangalore', 2500),
            (2, 'Dr. Abdul', 'Homeopathy', 'Fortis Hospital', 'Bangalore', 2000),
            (3, 'Dr. Shwetha', 'Homeopathy', 'KMC Hospital', 'Manipal', 1000),
            (4, 'Dr. Murphy', 'Dermatology', 'KMC Hospital', 'Manipal', 1500),
            (5, 'Dr. Farhana', 'Physician', 'Gleneagles Hospital', 'Bangalore', 1700),
            (6, 'Dr. Maryam', 'Physician', 'Gleneagles Hospital', 'Bangalore', 1500);
""")

In [13]:
duckdb.query("""
         SELECT d1.* FROM DOCTORS AS d1
         JOIN DOCTORS AS d2
         ON d1.id != d2.id AND d1.hospital = d2.hospital AND d1.speciality != d2.speciality
""").df()

,id,name,speciality,hospital,city,consultation_fee
0,3,Dr. Shwetha,Homeopathy,KMC Hospital,Manipal,1000
1,4,Dr. Murphy,Dermatology,KMC Hospital,Manipal,1500


In [14]:
duckdb.query("""
         SELECT d1.* FROM DOCTORS AS d1
         JOIN DOCTORS AS d2
         ON d1.id != d2.id AND d1.hospital = d2.hospital 
         ORDER BY d1.speciality
""").df()

,id,name,speciality,hospital,city,consultation_fee
0,4,Dr. Murphy,Dermatology,KMC Hospital,Manipal,1500
1,3,Dr. Shwetha,Homeopathy,KMC Hospital,Manipal,1000
2,5,Dr. Farhana,Physician,Gleneagles Hospital,Bangalore,1700
3,6,Dr. Maryam,Physician,Gleneagles Hospital,Bangalore,1500


### 5 - From the login_details table, fetch the users who logged in consecutively 3 or more times.

In [15]:
duckdb.query("""
            create table login_details(
            login_id int primary key,
            user_name varchar(50) not null,
            login_date date);
""")

In [16]:
duckdb.query("""
            insert into login_details values
            (101, 'Michael', current_date),
            (102, 'James', current_date),
            (103, 'Stewart', current_date+1),
            (104, 'Stewart', current_date+1),
            (105, 'Stewart', current_date+1),
            (106, 'Michael', current_date+2),
            (107, 'Michael', current_date+2),
            (108, 'Stewart', current_date+3),
            (109, 'Stewart', current_date+3),
            (110, 'James', current_date+4),
            (111, 'James', current_date+4),
            (112, 'James', current_date+5),
            (113, 'James', current_date+6);
""")

In [17]:
duckdb.query("""
            SELECT DISTINCT USER_NAME AS REPEATED_USERS
            FROM (SELECT *,
                  CASE WHEN USER_NAME = LEAD (USER_NAME, 1) OVER(ORDER BY LOGIN_DATE) 
                  AND  USER_NAME = LEAD (USER_NAME, 2) OVER(ORDER BY LOGIN_DATE)
                  THEN 1 END AS LEAD
                  FROM login_details) AS t1
                   
            WHERE LEAD IS NOT NULL
""").df()

,REPEATED_USERS
0,Stewart
1,James


### 6 - From the students table, write a SQL query to interchange the adjacent student names.

In [18]:
duckdb.query("""
            create table students
            (
            id int primary key,
            student_name varchar(50) not null
            );
            insert into students values
            (1, 'James'),
            (2, 'Michael'),
            (3, 'George'),
            (4, 'Stewart'),
            (5, 'Robin');
""")

In [19]:
duckdb.query("""
            SELECT * 
            FROM STUDENTS
""").df()

,id,student_name
0,1,James
1,2,Michael
2,3,George
3,4,Stewart
4,5,Robin


### 7 - From the weather table, fetch all the records when London had extremely cold temperature for 3 consecutive days or more.

In [20]:
duckdb.query("""
            create table weather
            (
            id int,
            city varchar(50),
            temperature int,
            day date
            );
            insert into weather values
            (1, 'London', -1, CAST('2021-01-01' AS date)),
            (2, 'London', -2, CAST('2021-01-02' AS date)),
            (3, 'London', 4, CAST('2021-01-03' AS date)),
            (4, 'London', 1, CAST('2021-01-04' AS date)),
            (5, 'London', -2, CAST('2021-01-05'AS date)),
            (6, 'London', -5, CAST('2021-01-06' AS date)),
            (7, 'London', -7, CAST('2021-01-07' AS date)),
            (8, 'London', 5, CAST('2021-01-08' AS date));
""")

In [21]:
duckdb.query("""
    SELECT *,
    CASE WHEN temperature < 0
         AND LEAD (TEMPERATURE, 1) OVER (ORDER BY DAY) < 0
         AND LEAD (TEMPERATURE, 2) OVER (ORDER BY DAY) < 0
         THEN 1 
         WHEN temperature < 0
         AND LEAD (TEMPERATURE, 1) OVER (ORDER BY DAY) < 0
         AND LAG (TEMPERATURE, 1) OVER (ORDER BY DAY) < 0
         THEN 1 
         WHEN temperature < 0
         AND LAG (TEMPERATURE, 1) OVER (ORDER BY DAY) < 0
         AND LAG (TEMPERATURE, 2) OVER (ORDER BY DAY) < 0
         THEN 1 ELSE 0 END AS FLAG
    FROM weather
""").df()

,id,city,temperature,day,FLAG
0,1,London,-1,2021-01-01,0
1,2,London,-2,2021-01-02,0
2,3,London,4,2021-01-03,0
3,4,London,1,2021-01-04,0
4,5,London,-2,2021-01-05,1
5,6,London,-5,2021-01-06,1
6,7,London,-7,2021-01-07,1
7,8,London,5,2021-01-08,0


### 8 - From the following 3 tables (event_category, physician_speciality, patient_treatment), write a SQL query to get the histogram of specialities of the unique physicians who have done the procedures but never did prescribe anything.

In [22]:
duckdb.query("""
            create table event_category
            (
              event_name varchar(50),
              category varchar(100)
            );

            create table physician_speciality
            (
              physician_id int,
              speciality varchar(50)
            );

            create table patient_treatment
            (
              patient_id int,
              event_name varchar(50),
              physician_id int
            );


            insert into event_category values ('Chemotherapy','Procedure');
            insert into event_category values ('Radiation','Procedure');
            insert into event_category values ('Immunosuppressants','Prescription');
            insert into event_category values ('BTKI','Prescription');
            insert into event_category values ('Biopsy','Test');


            insert into physician_speciality values (1000,'Radiologist');
            insert into physician_speciality values (2000,'Oncologist');
            insert into physician_speciality values (3000,'Hermatologist');
            insert into physician_speciality values (4000,'Oncologist');
            insert into physician_speciality values (5000,'Pathologist');
            insert into physician_speciality values (6000,'Oncologist');


            insert into patient_treatment values (1,'Radiation', 1000);
            insert into patient_treatment values (2,'Chemotherapy', 2000);
            insert into patient_treatment values (1,'Biopsy', 1000);
            insert into patient_treatment values (3,'Immunosuppressants', 2000);
            insert into patient_treatment values (4,'BTKI', 3000);
            insert into patient_treatment values (5,'Radiation', 4000);
            insert into patient_treatment values (4,'Chemotherapy', 2000);
            insert into patient_treatment values (1,'Biopsy', 5000);
            insert into patient_treatment values (6,'Chemotherapy', 6000);
""")

In [23]:
duckdb.query("""
            WITH t1 AS (SELECT PT.*, PS.speciality, EC.category
                        from patient_treatment as PT
                        JOIN physician_speciality as PS
                        ON PT.physician_id = PS.physician_id
                        JOIN event_category as EC
                        ON PT.event_name = EC.event_name),
                        
                 t2 AS (SELECT DISTINCT physician_id, category, speciality, event_name     
                        FROM t1
                        WHERE category != 'Prescription' AND category != 'Test' 
                        )
                        
            SELECT speciality,COUNT(speciality) AS speciality_count FROM t2
            GROUP BY speciality
""").df()

,speciality,speciality_count
0,Radiologist,1
1,Oncologist,3


### 9 - Find the top 2 accounts with the maximum number of unique patients on a monthly basis.

In [24]:
duckdb.query("""
--drop table patient_logs;
create table patient_logs
(
  account_id int,
  date date,
  patient_id int
);


insert into patient_logs values (1, CAST('2020-01-02' AS date), 100);
insert into patient_logs values (1, CAST('2020-01-27' AS date), 200);
insert into patient_logs values (2, CAST('2020-01-01' AS date), 300);
insert into patient_logs values (2, CAST('2020-01-21' AS date), 400);
insert into patient_logs values (2, CAST('2020-01-21' AS date), 300);
insert into patient_logs values (2, CAST('2020-01-01' AS date), 500);
insert into patient_logs values (3, CAST('2020-01-20' AS date), 400);
insert into patient_logs values (1, CAST('2020-03-04' AS date), 500);
insert into patient_logs values (3, CAST('2020-01-20' AS date), 450);

""")

In [25]:
duckdb.query("""
            WITH t1 as (SELECT account_id, MONTHNAME(date) AS month, patient_id, COUNT( DISTINCT patient_id) AS unique_patients, 
                        FROM patient_logs
                        GROUP BY month, patient_id, account_id),
                        
                 t2 AS (SELECT month, account_id, COUNT(unique_patients) as n_unique_patients,
                        FROM t1
                        GROUP BY month, account_id
                        ORDER BY n_unique_patients DESC),
                        
                 t3 AS (SELECT *, row_number () over (partition by month) as row_n
                        FROM t2)
                        
            SELECT month, account_id, n_unique_patients
            FROM t3
            WHERE row_n < 3
""").df()

,month,account_id,n_unique_patients
0,January,2,3
1,January,1,2
2,March,1,1


### 10 - SQL Query to fetch “N” consecutive records from a table with temperature lower than 0

In [26]:
duckdb.query("""
            --drop table weather2;
            create table weather2
            (
            id int,
            city varchar(50),
            temperature int,
            day date
            );
            insert into weather2 values
            (1,  'London',  -1, CAST('2021-01-01' AS date)),
            (2,  'London',  -2, CAST('2021-01-02' AS date)),
            (3,  'London',   4, CAST('2021-01-03' AS date)),
            (4,  'London',   1, CAST('2021-01-04' AS date)),
            (5,  'London',  -2, CAST('2021-01-05' AS date)),
            (6,  'London',  -5, CAST('2021-01-06' AS date)),
            (7,  'London',  -7, CAST('2021-01-07' AS date)),
            (8,  'London',   5, CAST('2021-01-08' AS date)),
            (9,  'London', -20, CAST('2021-01-09' AS date)),
            (10, 'London',  20, CAST('2021-01-10' AS date)),
            (11, 'London',  22, CAST('2021-01-11' AS date)),
            (12, 'London',  -1, CAST('2021-01-12' AS date)),
            (13, 'London',  -2, CAST('2021-01-13' AS date)),
            (14, 'London',  -2, CAST('2021-01-14' AS date)),
            (15, 'London',  -4, CAST('2021-01-15' AS date)),
            (16, 'London',  -9, CAST('2021-01-16' AS date)),
            (17, 'London',   0, CAST('2021-01-17' AS date)),
            (18, 'London', -10, CAST('2021-01-18' AS date)),
            (19, 'London', -11, CAST('2021-01-19' AS date)),
            (20, 'London', -12, CAST('2021-01-20' AS date)),
            (21, 'London', -11, CAST('2021-01-21' AS date));
""")

In [40]:
duckdb.query("""
            WITH t1 AS (SELECT *,
                        row_number() over() as rn,
                        id - (row_number() over()) as diff
                        FROM weather2
                        WHERE temperature < 0),
                        
                 t2 AS (SELECT *, COUNT (1) OVER (partition by diff) as n_of_days
                        FROM t1
                        ORDER BY n_of_days)
                        
            SELECT id, temperature, day
            FROM t2
            WHERE n_of_days = 3
""").df()

,id,temperature,day
0,5,-2,2021-01-05
1,6,-5,2021-01-06
2,7,-7,2021-01-07
